In [18]:
import io
from nltk import word_tokenize
from sentence_embedding import sentence_vec
import pandas as pd
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

### Understanding how embedding_dict will created

In [19]:
def understanding_load_vectors(fname):
    ## # taken from: https://fasttext.cc/docs/en/english-vectors.html
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        # tokens is the list where at 0th idx there is a word and after that there is vectors 
        tokens = line.rstrip().split(' ')
        # storing the word as key and its corresponding 300-dimensional vectors as float
        data[tokens[0]] = list(map(float, tokens[1:]))
        print(len(tokens[1:]))
        print(type(data), type(tokens))
        print(data)
        break

In [20]:
print("Loading embeddings.....")
embeddings = understanding_load_vectors("../input/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec")

Loading embeddings.....
300
<class 'dict'> <class 'list'>
{',': [0.1073, 0.0089, 0.0006, 0.0055, -0.0646, -0.06, 0.045, -0.0133, -0.0357, 0.043, -0.0356, -0.0032, 0.0073, -0.0001, 0.0258, -0.0166, 0.0075, 0.0686, 0.0392, 0.0753, 0.0115, -0.0087, 0.0421, 0.0265, -0.0601, 0.242, 0.0199, -0.0739, -0.0031, -0.0263, -0.0062, 0.0168, -0.0357, -0.0249, 0.019, -0.0184, -0.0537, 0.142, 0.06, 0.0226, -0.0038, -0.0675, -0.0036, -0.008, 0.057, 0.0208, 0.0223, -0.0256, -0.0153, 0.0022, -0.0482, 0.0131, -0.6016, -0.0088, 0.0106, 0.0229, 0.0336, 0.0071, 0.0887, 0.0237, -0.029, -0.0405, -0.0125, 0.0147, 0.0475, 0.0647, 0.0474, 0.0199, 0.0408, 0.0322, 0.0036, 0.035, -0.0723, -0.0305, 0.0184, -0.0026, 0.024, -0.016, -0.0308, 0.0434, 0.0147, -0.0457, -0.0267, -0.1703, -0.0099, 0.0417, 0.0235, -0.026, -0.1519, -0.0116, -0.0306, -0.0413, 0.033, 0.0723, 0.0365, -0.0001, 0.0042, 0.0346, 0.0277, -0.0305, 0.0784, -0.0404, 0.0187, -0.0225, -0.0206, -0.0179, -0.2428, 0.0669, 0.0523, 0.0527, 0.0149, -0.0708, -0.0

## Actual code

In [21]:
def load_vectors(fname):
    ## # taken from: https://fasttext.cc/docs/en/english-vectors.html
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        ## tokens is the list where at 0th idx there is a word and after that there is vectors  
        data[tokens[0]] = map(float, tokens[1:])
        
    return data

In [22]:
print("Loading embeddings.....")
embeddings = load_vectors("../input/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec")

Loading embeddings.....



KeyboardInterrupt



In [ ]:
embeddings[0:10]

In [ ]:
df = pd.read_csv("../input/IMDB_Dataset.csv")

df.sentiment = df.sentiment.apply(
    lambda x : 1 if x== "positive" else 0)

df = df.sample(frac=1).reset_index(drop=True)

print("Loading embeddings.....")
embeddings = load_vectors("../input/crawl-300d-2M.vec/crawl-300d-2M.vec")

print("Creating sentence vectors....")
vectors = []

for review in df.review.values:
    vectors.append(
        sentence_vec(
            s = review,
            embedding_dict = embeddings,
            stopwords = [],
            tokenizer = word_tokenize    
            )  
    )

# get numpy array of the list containing the sentence vectors
vectors = np.array(vectors)

y = df.sentiment.values

skf = model_selection.StratifiedKFold(n_splits=5)

for fold, (train_idx, test_idx) in enumerate(skf.split(X=vectors, y=y)):
    print("Training folds")
    x_train = vectors[train_idx, :]
    y_train = y[train_idx, :]
    
    x_valid = vectors[test_idx, :]
    y_valid = y[test_idx, :]
    
    model = linear_model.LogisticRegression()
    
    model.fit(x_train, y_train)
    
    preds = model.predict(x_valid)
    
    accuracy = metrics.accuracy_score(preds, y_valid)
    print("Accuracy---", accuracy)
    print(" ")